<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 2



### 2.1 Linear Regression in 1D Forward Prediction



#### Linear Regression



$\hat y =b+wx$



In [ ]:
import torch



In [ ]:
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(-1.0, requires_grad=True)



In [ ]:
def forward(x):
  y = w * x + b
  return y



In [ ]:
x = torch.tensor([1.0])



In [ ]:
yhat = forward(x)
yhat



$x =
  \begin{pmatrix}
    1 \\
    2
  \end{pmatrix}$



In [ ]:
x = torch.tensor(
    [
        [1.0],
        [2.0]
    ],
)



In [ ]:
forward(x)



#### `Linear` class



In [ ]:
import torch.nn

torch.manual_seed(1)



$\hat y_{1 \times 1} =b_{1 \times 1}+w_{1 \times 1}x_{1 \times 1}$



In [ ]:
model = torch.nn.Linear(in_features=1, out_features=1)



$b_{1 \times 1}$ and $w_{1 \times 1}$



In [ ]:
for p in model.parameters():
  print(p)



In [ ]:
x = torch.tensor([0.0])
yhat = model(x)
yhat



$x =
  \begin{pmatrix}
    1 \\
    2
  \end{pmatrix}$



In [ ]:
x = torch.tensor(
    [
        [1.0],
        [2.0]
    ]
)



In [ ]:
yhat = model(x)
yhat



#### Custom Modules
* may include multiple models


In [ ]:
import torch.nn



In [ ]:
class LR(torch.nn.Module):
  def __init__(self, in_size, output_size):
    super(LR, self).__init__()
    self.linear = torch.nn.Linear(in_size, output_size)

  def forward(self, x):
    return self.linear(x)



In [ ]:
model = LR(1, 1)

state = model.state_dict()

weight = state['linear.weight']
weight.data[0] = torch.tensor([0.5153])

bias = state['linear.bias']
bias.data[0] = torch.tensor([-0.4414])



$b_{1 \times 1}$ and $w_{1 \times 1}$



In [ ]:
for p in model.parameters():
  print(p)



In [ ]:
x = torch.tensor([1.0])
model(x)



In [ ]:
x = torch.tensor(
    [
        [1.0],
        [2.0]
    ]
)
model(x)



In [ ]:
state = model.state_dict()
state



### 2.2 Linear Regression in 1D Training



#### Linear Regression



Cost function
$$
l(w, b)=\frac{1}{N}
  \sum_{n=1}^N
    \left(
      y_n - \left(
          wx_n + b
        \right)
    \right)^2
$$



#### Gradient Descent
$$
w^{k+1}=w^{k}-η \frac{d}{dw}l\left(w^k\right)
$$

symbol | description
:------:|-----
$w$ | weight
$l(w)$ | loss as a function of weigth
$$\frac{d}{dw}l(w)$$ | slope of loss as a function of weigth
$k$ | training step
$\eta$ | **learning rate**



#### PyTorch Slope



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch



In [ ]:
w = torch.tensor(-10.0, requires_grad=True)



In [ ]:
X = torch.linspace(-3, 3, 61).view(-1, 1)



In [ ]:
f = -3 * X



In [ ]:
Y = f + 0.1 * torch.randn(X.size())



In [ ]:
def plot_forward(X, f, Y, flabel='truth', Ylabel='measurements'):
  try:
    plt.plot(X.numpy(), f.numpy(), label=flabel)
  except RuntimeError:
    plt.plot(X.numpy(), f.detach().numpy(), label=flabel)

  try:
    plt.plot(X.numpy(), Y.numpy(), '.', label=Ylabel)
  except RuntimeError:
    plt.plot(X.numpy(), Y.detach().numpy(), '.', label=Ylabel)

  plt.xlabel('X')
  plt.ylabel('f')
  plt.legend(loc=0)
  plt.grid(True)

plot_forward(X, f, Y)



In [ ]:
def forward(x):
  return w * x



In [ ]:
def criterion(yhat, y):
  return torch.mean((yhat - y) ** 2)



In [ ]:
def plot_cost(weight, cost):
  plt.plot(weight, cost, '.-')
  plt.xlabel('w')
  plt.ylabel('cost')
  plt.grid(True)



In [ ]:
# learning rate
lr = 0.1
lr_neg = -lr

weight = []
cost = []

# epoch : training steps
for epoch in range(4):
  Yhat = forward(X)

  loss = criterion(Yhat, Y)
  loss.backward()

  w.data += (lr_neg) * w.grad.data
  w.grad.data.zero_()

  plt.figure()

  plt.subplot(2, 1, 1)
  weight.append(w.data.item())
  cost.append(loss.item())
  plot_cost(weight, cost)

  plt.subplot(2, 1, 2)
  plot_forward(X, Yhat, Y, 'prediction', 'data')



#### Cost Surface



in PyTorch, the hard way



In [ ]:
def forward(x):
  return w * x + b



cost function



In [ ]:
def criterion(yhat, y):
  return torch.mean((yhat-y) ** 2)



Initialize tensors



In [ ]:
w = torch.tensor(-15.0, requires_grad=True)
b = torch.tensor(-10.0, requires_grad=True)
X = torch.linspace(-3.0, 3.0, 61).view(-1, 1)
f = 1.0 * X - 1.0
Y = f + 0.1 * torch.randn(X.size())



Check sizes
$$
\begin{align}
  \hat y &= wx+b \\
  {\hat y}_{pq \times n} &= \begin{pmatrix}
    w_{p q \times 1} & b_{p q \times 1}
  \end{pmatrix}_{pq \times 2}
  \begin{pmatrix}
    x_{1 \times n} \\ 1_{1 \times n}
  \end{pmatrix}_{2 \times n} \\
error_{p q \times n}&={\hat y}_{pq \times n} - 1_{pq \times 1} y_{1 \times n}
\end{align}
$$



In [ ]:
def plot_cost_surface(
    w_range:int, b_range:int,
    X:torch.tensor, Y:torch.tensor,
    n_samples:int=31, ax:plt.axis=plt.gca(),
  ):
  w_vec = np.linspace(-w_range, w_range, n_samples)
  b_vec = np.linspace(-b_range, b_range, n_samples)

  w_grid, b_grid = np.meshgrid(w_vec, b_vec)

  x = X.numpy().reshape(1, -1)
  y = Y.numpy().reshape(1, -1)

  x_one = np.vstack([
    x,
    np.ones_like(x)
  ])

  w_flat = w_grid.flatten()
  b_flat = b_grid.flatten()
  wb = np.column_stack([w_flat, b_flat])

  assert wb.shape[-1] == x_one.shape[0], (
      '\n'
      f"w_flat.shape = {w_flat.shape}\n"
      f"wb.shape = {wb.shape}\n"
      f"x_one.shape = {x_one.shape}\n"
  )
  yhat = wb @ x_one

  ones_y = np.ones((len(w_flat), 1))

  # using numpy broadcasting along the first dimension
  # yhat [pq, n]
  # y[1, n]
  error = yhat - y
  z_flat = np.mean(error**2, axis=1)

  Z = z_flat.reshape(*w_grid.shape)

  ax.contour(w_grid, b_grid, Z)


plot_cost_surface(15, 15, X, Y)
plt.xlabel('w')
plt.ylabel('b')
plt.axis('equal')
plt.grid(True)



In [ ]:
lr = 0.1
lr_neg = -lr

w_list = []
b_list = []
loss_list = []

ax = plt.gca()
plot_cost_surface(15, 15, X, Y, ax=ax)

for epoch in range(15):
  Yhat = forward(X)
  loss = criterion(Yhat, Y)

  loss.backward()

  ax.plot([w.data.item()], [b.data.item()], '.')
  ax.plot(
      [w.data.item(), w.data.item()+(lr_neg)*w.grad.data.item()],
      [b.data.item(), b.data.item()+(lr_neg)*b.grad.data.item()],
      '-'
  )

  w.data += (lr_neg) * w.grad.data
  w.grad.data.zero_()

  b.data += (lr_neg) * b.grad.data
  b.grad.data.zero_()

  w_list.append(w.data.item())
  b_list.append(b.data.item())
  loss_list.append(loss.data.item())

ax.set_xlabel('w')
ax.set_ylabel('b')
ax.grid(True)



### 3.1 Stochastic Gradient Descent & the Data Loader



Truth & Measurement



In [36]:
w = torch.tensor(-15.0, requires_grad=True)
b = torch.tensor(-10.0, requires_grad=True)

X = torch.linspace(-3, 3, 61).view(-1, 1)
f = (-3) * X
Y = f + 0.1 * torch.randn(X.size())

plt.plot(X.numpy(), f.numpy(), label='f')
plt.plot(X.numpy(), Y.numpy(), '.', label='Y')
plt.xlabel('x')
plt.ylabel('f')
plt.legend(loc=0)

plt.grid(True)



Error in callback <function flush_figures at 0x7cd2fe110280> (for post_execute):


KeyboardInterrupt: 

Stochastic Gradient Descent updates weights at each data point



In [ ]:
# learning rate
lr = 0.1
lr_neg = -lr

weight = []
cost = []

w_list = []
b_list = []

# epoch : training steps
for epoch in range(4):

  total = 0

  for x, y in zip(X, Y):
    yhat = forward(x)

    loss = criterion(yhat, y)
    loss.backward()

    w.data += (lr_neg) * w.grad.data
    b.data += (lr_neg) * b.grad.data

    w_list.append(w.data.item())
    b_list.append(b.data.item())

    w.grad.data.zero_()
    b.grad.data.zero_()

    total += loss.item()

    weight.append(w.data.item())
    cost.append(loss.item())

  cost.append(total)

  plt.figure()

  plt.subplot(1, 2, 1)
  plot_forward(X, forward(X), Y, 'prediction', 'data')

  plt.subplot(1, 2, 2)
  plt.plot(w_list, b_list, '.')
  plt.plot(w_list[0], b_list[0], 'o')
  plt.xlabel('w')
  plt.ylabel('b')
  plt.grid(True)



#### Data Loader



In [ ]:
import torch.utils.data



In [ ]:
class Data(torch.utils.data.Dataset):
  def __init__(self):
    self.x = torch.linspace(-3, 3, 61).view(-1, 1)
    self.y = -3 * X + 1
    self.len = self.x.shape[0]

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.len


In [ ]:
dataset = Data()



First element



In [ ]:
x, y = dataset[0]



In [ ]:
x



In [ ]:
y



Slicing



In [ ]:
x, y = dataset[:3]



In [ ]:
x



In [ ]:
y

